# Collect META5 Risk Variants Stats From Other GWAS
- **Author** - Frank Grenn
- **Date Started** - November 2019
- **Quick Description:** code to gather summary stats for the meta5 risk loci in other gwases. **Note** that this list is manually edited as the end to make sure maf is consistent  

 - #### 1) Age of Onset GWAS
 - #### 2) GBA Age of Onset GWAS
 - #### 3) GBA Modifier GWAS
 - #### 4) LRRK2 Modifier GWAS
 - #### 5) Asian GWAS
 - #### 6) META5 GWAS


In [ ]:
import pandas as pd
from scipy.stats import norm

In [ ]:
DATADIR ='/path/to/AppDataProcessing' 
WRKDIR = f"{DATADIR}/othersummarystats"

In [ ]:
writer = pd.ExcelWriter(f'{WRKDIR}/other_sum_stats.xlsx', engine='xlsxwriter')

#### get the meta5 loci

In [ ]:
meta5_loci = pd.read_csv(f"{DATADIR}/META5Loci.csv")
meta5_loci['GWAS']='META5'
print(meta5_loci.shape)
print(meta5_loci.head())

In [ ]:
prog_loci = pd.read_csv(f"{DATADIR}/ProgressionLoci.csv")
prog_loci['GWAS']='Progression'
prog_loci['EFFECT_ALLELE']=prog_loci['ALT']
prog_loci['OTHER_ALLELE']=prog_loci['REF']
prog_loci['P_COJO']=None
prog_loci = prog_loci.rename(columns={"MAF": "FREQ"})
prog_loci['EFFECT_FREQ']=prog_loci['FREQ']

print(prog_loci.shape)
print(prog_loci.head())

In [ ]:
asian_loci = pd.read_csv(f"{DATADIR}/AsianLoci.csv")
asian_loci['GWAS']='Asian'
asian_loci['EFFECT_ALLELE']=asian_loci['ALT']
asian_loci['OTHER_ALLELE']=asian_loci['REF']
asian_loci['P_COJO']=None
asian_loci = asian_loci.rename(columns={"MAF": "FREQ"})
asian_loci['EFFECT_FREQ']=asian_loci['FREQ']
print(asian_loci.shape)
print(asian_loci.head())

In [ ]:
loci = meta5_loci.append(prog_loci).append(asian_loci)
print(loci.shape)
print(loci.head())

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE']]

## 1) Age of Onset GWAS

In [ ]:
aoo = pd.read_csv("/path/to/age_of_onset/sorted_AAO_april3_18_final_all_data.txt", sep="\t")
print(aoo.shape)
print(aoo.head())

In [ ]:
print(list(aoo.columns.values))

In [ ]:
aoo_sub = aoo[['MarkerName','Allele1','Allele2','Freq1','Effect','StdErr','P-value']]

In [ ]:
loci_sub['MarkerName'] = 'chr' + loci_sub['CHR_BP']
print(loci_sub.head())

In [ ]:
merge_aoo = pd.merge(loci_sub, aoo_sub, on='MarkerName',how='left')
print(merge_aoo.shape)
print(merge_aoo.head())

#### now check for the right allele
looks like Allele1 from aoo should match EFFECT_ALLELE

In [ ]:
aoo_match=merge_aoo[merge_aoo['EFFECT_ALLELE']==merge_aoo['Allele1'].str.upper()]
print(aoo_match.shape)
print(aoo_match.head())

In [ ]:
aoo_mismatch=merge_aoo[merge_aoo['EFFECT_ALLELE']!=merge_aoo['Allele1'].str.upper()]
print(aoo_mismatch.shape)
print(aoo_mismatch.head())

In [ ]:
aoo_mismatch['Allele1']=aoo_mismatch['EFFECT_ALLELE']
aoo_mismatch['Allele2']=aoo_mismatch['OTHER_ALLELE']
aoo_mismatch['Freq1']=1-aoo_mismatch['Freq1']
aoo_mismatch['Effect']=-1*aoo_mismatch['Effect']


In [ ]:
final_aoo=aoo_mismatch.append(aoo_match)
print(final_aoo.shape)
print(final_aoo.head())

In [ ]:
final_aoo = final_aoo[['RSID','CHR','BP','CHR_BP','Freq1','Effect','StdErr','P-value']]
final_aoo.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_aoo.shape)
print(final_aoo.head())

In [ ]:
final_aoo.to_excel(writer, sheet_name='AAO',index=None)

## 2) GBA Age of Onset GWAS

In [ ]:
gba_aoo = pd.read_csv("/path/to/GBA_GWAS/HARDCALL_GBA_case_age.SingleWald_MAF.assoc", sep="\t")
print(gba_aoo.shape)
print(gba_aoo.head())

In [ ]:
gba_aoo_sub = gba_aoo[['Test','A1','A2','CHROM','POS','Beta','SE','Pvalue','MAF']]
print(gba_aoo_sub.shape)

In [ ]:
merge_gba_aoo = pd.merge(loci_sub, gba_aoo_sub, how='left', left_on='CHR_BP', right_on='Test')
print(merge_gba_aoo.shape)
print(merge_gba_aoo.head())

#### now check for the right allele
looks like A1 from gba_aoo should match EFFECT_ALLELE

In [ ]:
gba_aoo_match=merge_gba_aoo[merge_gba_aoo['EFFECT_ALLELE']==merge_gba_aoo['A1']]
print(gba_aoo_match.shape)
print(gba_aoo_match.head())

In [ ]:
gba_aoo_mismatch=merge_gba_aoo[merge_gba_aoo['EFFECT_ALLELE']!=merge_gba_aoo['A1']]
print(gba_aoo_mismatch.shape)
print(gba_aoo_mismatch.head())

In [ ]:
gba_aoo_mismatch['MAF']=1-gba_aoo_mismatch['MAF']
gba_aoo_mismatch['Beta']=-1*gba_aoo_mismatch['Beta']

In [ ]:
final_gba_aoo=gba_aoo_mismatch.append(gba_aoo_match)
print(final_gba_aoo.shape)
print(final_gba_aoo.head())

In [ ]:
final_gba_aoo = final_gba_aoo[['RSID','CHR','BP','CHR_BP','MAF','Beta','SE','Pvalue']]
final_gba_aoo.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
final_gba_aoo.to_excel(writer, sheet_name='GBA_AAO',index=None)

## 3) GBA Modifier GWAS

In [ ]:
meta_gba = pd.read_csv("/path/to/GBA_GWAS/META_GBA_penetrance_modifier_carriers_GWAS.txt", sep="\t")
print(meta_gba.shape)
print(meta_gba.head())

In [ ]:
meta_gba_sub = meta_gba[['MarkerName','Allele1','Allele2','Freq1','StdErr','Effect','P-value']]
print(meta_gba_sub.shape)

In [ ]:
merge_gba = pd.merge(loci_sub, meta_gba_sub, how='left', left_on='CHR_BP', right_on='MarkerName')
print(merge_gba.shape)
print(merge_gba.head())

#### now check for the right allele
looks like Allele1 from gba shoul match EFFECT_ALLELE

In [ ]:
gba_match=merge_gba[merge_gba['EFFECT_ALLELE']==merge_gba['Allele1'].str.upper()]
print(gba_match.shape)
print(gba_match.head())

In [ ]:
gba_mismatch=merge_gba[merge_gba['EFFECT_ALLELE']!=merge_gba['Allele1'].str.upper()]
print(gba_mismatch.shape)
print(gba_mismatch.head())

In [ ]:
gba_mismatch['Freq1']=1-gba_mismatch['Freq1']
gba_mismatch['Effect']=-1*gba_mismatch['Effect']

In [ ]:
final_gba = gba_match.append(gba_mismatch)
print(final_gba.shape)
print(final_gba.head())

In [ ]:
final_gba = final_gba[['RSID','CHR','BP','CHR_BP','Freq1','Effect','StdErr','P-value']]
final_gba.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:

print(final_gba.head())

In [ ]:
final_gba.to_excel(writer, sheet_name='GBA_mod',index=None)

## 4) LRRK2 Modifier GWAS

In [ ]:
lrrk2 = pd.read_csv(f"{WRKDIR}/LRRK2_GWAS_risk_Variants.csv")
print(lrrk2.shape)
print(lrrk2.head())

In [ ]:
lrrk2['name']=lrrk2['name'].astype(str).str[:-2]
print(lrrk2.head())

In [ ]:
merge_lrrk2 = pd.merge(loci_sub, lrrk2, how='left', left_on='RSID', right_on='name')
print(merge_lrrk2.shape)
print(merge_lrrk2.head())

#### now check for the right allele
looks like A1 from lrrk2 shoul match EFFECT_ALLELE

In [ ]:
lrrk2_match=merge_lrrk2[merge_lrrk2['EFFECT_ALLELE']==merge_lrrk2['A1']]
print(lrrk2_match.shape)
print(lrrk2_match.head())

In [ ]:
lrrk2_mismatch=merge_lrrk2[merge_lrrk2['EFFECT_ALLELE']!=merge_lrrk2['A1']]
print(lrrk2_mismatch.shape)
print(lrrk2_mismatch.head())

In [ ]:
lrrk2_mismatch['A1_freq']=1-lrrk2_mismatch['A1_freq']
lrrk2_mismatch['beta']=-1*lrrk2_mismatch['beta']

In [ ]:
final_lrrk2 = lrrk2_match.append(lrrk2_mismatch)
print(final_lrrk2.shape)
print(final_lrrk2.head())

In [ ]:
final_lrrk2 = final_lrrk2[['RSID','CHR','BP','CHR_BP','A1_freq','beta','std.error','p.value']]
final_lrrk2.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_lrrk2.head())

In [ ]:
final_lrrk2.to_excel(writer, sheet_name='LRRK2',index=None)

## 5) Asian GWAS

In [ ]:
#6724PDcases-24851controls-5843213snps-summary-stats-maf5-metaP-SE.txt.gz
#6724PDcases-24851controls-5843213snps-summary-stats-metaP-SE.txt.gz
#6724PDcases-24851controls-5843213snps-summary-stats-metaP.txt.gz
#779PDcases-13227controls-2402394snps-hmg2017-summary-stats-OR-SE.txt
#Asian_GWAS_2019.gz

asian_gwas = pd.read_csv(f"{DATADIR}/asiangwas_sumstats_harmonized.csv")
print(asian_gwas.shape)
print(asian_gwas.head())


In [ ]:
asian_gwas_sub = asian_gwas[['RSID','A1','A2','BETA','SE','FREQ','P']]

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE','BETA']]

In [ ]:
merge_asian_gwas = pd.merge(loci_sub, asian_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_asian_gwas.shape)
print(merge_asian_gwas.tail())

In [ ]:
merge_asian_gwas

#### now check for the right allele
need to look at beta since we don't have freqs, looks like asian gwas A1 should match EFFECT_ALLELE

In [ ]:
asian_gwas_match=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']==merge_asian_gwas['A1']]
print(asian_gwas_match.shape)
print(asian_gwas_match.head())

In [ ]:
asian_gwas_mismatch=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']!=merge_asian_gwas['A1']]
print(asian_gwas_mismatch.shape)
print(asian_gwas_mismatch.tail())

In [ ]:
asian_gwas_mismatch['BETA_y']=-1*asian_gwas_mismatch['BETA_y']

In [ ]:
final_asian_gwas = asian_gwas_match.append(asian_gwas_mismatch)
print(final_asian_gwas.shape)
print(final_asian_gwas.head())

In [ ]:
#for now no frequencies are available so set to NA
final_asian_gwas['maf']=None

#### (optional) recalculate the SE if missing

P = 2 * pnorm(-ABS(Z))  
Z = beta/SE  
in python:  
P = 2 * norm.cdf(-abs(beta/SE))  
or  
SE=beta/norm.ppf(P/2)

In [ ]:
no_se = final_asian_gwas[final_asian_gwas['SE']=='#NUM!']
no_se

In [ ]:
rsid_to_calc = 'rs11557080'#rs11557080

In [ ]:
beta = final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]['BETA_y']
P = final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]['P']
final_asian_gwas['SE'][final_asian_gwas['RSID']==rsid_to_calc] = abs(beta/-norm.ppf(P/2))

In [ ]:
final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]

In [ ]:
final_asian_gwas = final_asian_gwas[['RSID','CHR','BP','CHR_BP','maf','BETA_y','SE','P']]
final_asian_gwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:

print(final_asian_gwas.shape)
print(final_asian_gwas.tail(10))


In [ ]:
final_asian_gwas.to_excel(writer, sheet_name='Asian_GWAS',index=None)

## 6) META5 GWAS

In [ ]:
meta5_gwas = pd.read_csv(f"{DATADIR}/meta5_sumstats_harmonized.csv")
print(meta5_gwas.shape)
print(meta5_gwas.head())

In [ ]:
loci_sub.columns

In [ ]:
meta5_gwas_sub = meta5_gwas[['RSID','REF','ALT','A1','A2','BETA','SE','FREQ','P']]

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE']]

In [ ]:
merge_meta5_gwas = pd.merge(loci_sub, meta5_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_meta5_gwas.shape)
print(merge_meta5_gwas.tail())

In [ ]:
print(merge_meta5_gwas.head())

#### now check for the right allele
need to look at beta since we don't have freqs, looks like asian gwas A1 should match EFFECT_ALLELE

In [ ]:
meta5_gwas_match=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']==merge_meta5_gwas['A1']]
print(meta5_gwas_match.shape)
print(meta5_gwas_match.head())

In [ ]:
meta5_gwas_mismatch=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']!=merge_meta5_gwas['A1']]
print(meta5_gwas_mismatch.shape)
print(meta5_gwas_mismatch.tail())

In [ ]:
meta5_gwas_mismatch['BETA']=-1*meta5_gwas_mismatch['BETA']
meta5_gwas_mismatch['FREQ']=1-meta5_gwas_mismatch['FREQ']

In [ ]:
final_meta5_gwas = meta5_gwas_match.append(meta5_gwas_mismatch)

print(final_meta5_gwas.shape)
print(final_meta5_gwas.head())

In [ ]:
final_meta5_gwas = final_meta5_gwas[['RSID','CHR','BP','CHR_BP','FREQ','BETA','SE','P']]
final_meta5_gwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
final_meta5_gwas.to_excel(writer, sheet_name='META5',index=None)

In [ ]:
writer.save()